In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Load base Qwen model
base_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B", trust_remote_code=True).to(device)

# 2. Load LoRA adapter
model = PeftModel.from_pretrained(base_model, "BirdieByte1024/Qwen2.5-1.5B-LoRA-dental").to(device)
model.eval()

# 3. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B", trust_remote_code=True)

# 4. Example prompt
prompt = """### Instruction:
Given an ADA dental procedure code and its short name, provide a detailed explanation that helps a patient understand what the procedure involves.

### Code:
39 - CT of facial bone

### Response:"""

inputs = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=False,
        temperature=0.7,
        top_p=0.95
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


### Instruction:
Given an ADA dental procedure code and its short name, provide a detailed explanation that helps a patient understand what the procedure involves.

### Code:
39 - CT of facial bone

### Response: 
Computerized tomography of facial bone; includes both anterior and posterior views; includes both soft and hard tissue; includes both bone and soft tissue; includes both anterior and posterior views; includes both soft and hard tissue; includes both bone and soft tissue; includes both anterior and posterior views; includes both soft and hard tissue; includes both bone and soft tissue; includes both anterior and posterior views; includes both soft and hard tissue; includes both bone and soft tissue; includes both anterior and posterior views; includes both soft and hard tissue; includes both bone and soft tissue; includes both anterior and posterior views; includes both soft and hard tissue; includes both bone and soft tissue; includes both anterior and posterior views; includ

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prompts
base_prompt = "Given an ADA dental procedure code and its short name, provide a detailed explanation that helps a patient understand what the procedure involves. Explain the meaning of ADA dental code 39: CT of facial bone."

lora_prompt = """### Instruction:
Given an ADA dental procedure code and its short name, provide a detailed explanation that helps a patient understand what the procedure involves.

### Code:
39 - CT of facial bone

### Response:"""

# ========== BASE MODEL ==========
print("\n🔹 BASE MODEL (Qwen/Qwen2.5-1.5B)\n" + "-"*60)
print(f"📝 Input Prompt:\n{base_prompt}\n")

# Load model/tokenizer
base_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B", trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B", trust_remote_code=True).to(device).eval()

# Generate
inputs = base_tokenizer(base_prompt, return_tensors="pt").to(device)
with torch.no_grad():
    base_outputs = base_model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.7,
        top_p=0.95
    )
base_response = base_tokenizer.decode(base_outputs[0], skip_special_tokens=True)
print(f"💬 Response:\n{base_response}\n")


# ========== LoRA MODEL ==========
print("\n🔸 LoRA MODEL (BirdieByte1024/Qwen2.5-1.5B-LoRA-dental)\n" + "-"*60)
print(f"📝 Input Prompt:\n{lora_prompt}\n")

# Load LoRA
lora_base = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B", trust_remote_code=True).to(device)
lora_model = PeftModel.from_pretrained(lora_base, "BirdieByte1024/Qwen2.5-1.5B-LoRA-dental").to(device).eval()

inputs = base_tokenizer(lora_prompt, return_tensors="pt").to(device)
with torch.no_grad():
    lora_outputs = lora_model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.7,
        top_p=0.95
    )
lora_response = base_tokenizer.decode(lora_outputs[0], skip_special_tokens=True)
print(f"💬 Response:\n{lora_response}\n")


🔹 BASE MODEL (Qwen/Qwen2.5-1.5B)
------------------------------------------------------------
📝 Input Prompt:
Given an ADA dental procedure code and its short name, provide a detailed explanation that helps a patient understand what the procedure involves. Explain the meaning of ADA dental code 39: CT of facial bone.



Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


💬 Response:
Given an ADA dental procedure code and its short name, provide a detailed explanation that helps a patient understand what the procedure involves. Explain the meaning of ADA dental code 39: CT of facial bone. ADA dental code 39: CT of facial bone is a diagnostic procedure that involves the use of computed tomography (CT) to obtain detailed images of the facial bones. This procedure is used to diagnose and evaluate various conditions affecting the facial bones, such as fractures, tumors, infections, and other abnormalities. The CT scan provides a three-dimensional view of the facial bones, which can help doctors identify the location, size, and shape of the bone abnormalities. The procedure is typically performed by a radiologist or a dentist who specializes in oral and maxillofacial radiology. The patient is usually asked to lie still on a table while the CT scanner rotates around the head and takes multiple images. The images are then analyzed by a radiologist to


🔸 LoRA 

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


💬 Response:
### Instruction:
Given an ADA dental procedure code and its short name, provide a detailed explanation that helps a patient understand what the procedure involves.

### Code:
39 - CT of facial bone

### Response: 
Computerized tomography of facial bone; includes both anterior and posterior views; includes both soft and hard tissue; includes both bone and soft tissue; includes both anterior and posterior views; includes both soft and hard tissue; includes both bone and soft tissue; includes both anterior and posterior views; includes both soft and hard tissue; includes both bone and soft tissue; includes both anterior and posterior views; includes both soft and hard tissue; includes both bone and soft tissue; includes both anterior and posterior views; includes both soft and hard tissue; includes both bone and soft tissue; includes both anterior and posterior views; includes both soft and hard tissue; includes both bone and soft tissue; includes both anterior and posterior v